In [ ]:
from keras.models import load_model
model = load_model('whale.flukes.4250_classes.weights.best.hdf5')

target_size = (150, 150)

# Classify new_whale image

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1. / 255)

batch_size = 128
workes = 8

test_generator = test_datagen.flow_from_directory(
    "data/test",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical', shuffle=False)

score = model.evaluate_generator(
    generator=test_generator)

print("Loss: ", score[0], "Accuracy: ", score[1])

scores = model.predict_generator(
    generator=test_generator)

In [ ]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(test_generator.classes, list(map(lambda x: np.argmax(x), scores)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20, 20))
plt.imshow(cnf_matrix, interpolation='none')
plt.colorbar()
plt.show()

In [ ]:
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
import cv2
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

def path_to_tensor(img_path, target_size):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=target_size)
    # convert PIL.Image.Image type to 3D tensor with shape (size, size, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, size, size, colors) and return 4D tensor
    return np.expand_dims(x, axis=0)

def whale_id(generator, predicted_index):
    return [key for key, value in test_generator.class_indices.items() if value == predicted_index][0]

def display_image(path):
    if path is None:
        return
    
    img = cv2.imread(path)
   
    # convert BGR image to RGB for plotting
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # display the image, along with bounding box
    plt.imshow(cv_rgb)
    plt.show()
    
classes = model.output.shape[1]

# new whales
file = 'fa2b1d93.jpg' 
file = 'f6968c37.jpg'
file = 'f17f76c3.jpg'

# labaled whales
# aa1569e8.jpg w_f22f3e3
# file = 'aa1569e8.jpg'

input_img_path = f'train/{file}'
img = path_to_tensor(input_img_path, target_size)
img = preprocess_input(img)
print('Input image')
display_image(input_img_path)

result = model.predict(img)

predicated_index = np.argmax(result)
whale_id = whale_id(test_generator, predicated_index)

print(f'Predicted whale id: {whale_id}')

print('Top 5 indexes and probabilities')
_, sorted_classes = zip(*sorted(zip(result[0], range(classes)), reverse=True))
probs = np.array(result[0])
print(sorted_classes[:5])
print(probs[np.array(sorted_classes[:5])])

df = pd.read_csv("train.csv", header = 0)
df = df[df.Id == whale_id]

print('\nTop predicted whale images')

for file in df.Image.values:
    predicated_image_path = f'train/{file}'
    print(predicated_image_path)
    display_image(predicated_image_path)


In [ ]:
import pandas as pd
import pathlib
import shutil
from keras.preprocessing.image import ImageDataGenerator

df = pd.read_csv("train.csv", header = 0)

df = df[df.Id == 'new_whale']

dst_dir = './data/predict/new_whale'

pathlib.Path(dst_dir).mkdir(parents=True, exist_ok=True)

for idx, file in enumerate(df.Image):
    shutil.copy(f'./train/{file}', dst_dir)

datagen = ImageDataGenerator(rescale=1. / 255)

new_whale_generator = datagen.flow_from_directory(
    "data/predict",
    target_size=target_size,
    batch_size=128,
    class_mode=None, 
    shuffle=False)

new_whale_scores = model.predict_generator(
    generator=new_whale_generator,
    workers=8)

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = datagen.flow_from_directory(
    "data/train",
    target_size=target_size,
    batch_size=128,
    class_mode=None, 
    shuffle=False)

test_scores = model.predict_generator(
    generator=test_generator,
    workers=8)

# What My Deep Model Doesn't Know...

http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html

In [ ]:
import numpy as np

for i in new_whale_scores[:10]:
    print(sorted(i,reverse=True)[:10])
    print(f'Predictive mean: {np.mean(i, axis=0)}')
    print(f'Predictive variance: {np.var(i, axis=0)}')
    print(f'Predictive std: {np.std(i)}')